In [67]:
import numpy as np
import json
import random
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import sys

sys.path.append('../src')

from config.config import PATHS

In [68]:
# No. of agents
N = 7
# No. of available spots
s = 5

In [69]:
import matplotlib.colors as mcolors
named_colors = mcolors.CSS4_COLORS
color_names = list(named_colors.keys())
random.seed(1234)
random.shuffle(color_names)
shape_names = ['dot','diamond','square','triangle','star','triangleDown']

def bin_to_decimal(bin_str):
    return int(bin_str, 2)

def get_color(bin_str):
    decimal = bin_to_decimal(bin_str)
    return color_names[decimal % len(color_names)]

def get_shape(bin_str):
    decimal = bin_to_decimal(bin_str)
    return shape_names[decimal % len(shape_names)]

In [70]:
file_name = PATHS['graphs'] / f'graph_data_N{N:d}s{s:d}.json'
with open(file_name, 'r') as f:
  struct = json.load(f)
#print(json.dumps(struct, indent = 2))

In [71]:
Npat = 2
edges = []
e_colors = {}
n_colors = {}
n_shapes = {}
for n1 in struct[Npat]:
    n_colors[n1] = get_color(''.join(i for i in struct[Npat][n1]["pattern"]))
    n_shapes[n1] = get_shape(''.join(i for i in struct[Npat][n1]["pattern"]))
    for n2 in struct[Npat][str(n1)]["neigh"]:  
        if len(struct[Npat][str(n1)]["strat"]) > 1:
            edges.append((n2, n1))

            copy_strat = False
            if len(struct[Npat][str(n1)]["strat"]) == 2:
                copy_strat = (struct[Npat][str(n1)]["strat"]['0'] == '0') and (struct[Npat][str(n1)]["strat"]['1'] == '1')
                if copy_strat:
                    e_colors[f'{n2}-{n1}'] = 'darkturquoise'
                else:
                    e_colors[f'{n2}-{n1}'] = 'black'
            else:
                e_colors[f'{n2}-{n1}'] = 'black'

In [72]:
# print(edges)
# print(e_colors)
# print(n_colors)

In [73]:
n_colors['3']

'darkorchid'

In [74]:
DG = nx.DiGraph()
DG.add_nodes_from([str(i) for i in range(N)])
DG.add_edges_from(edges)
for node in DG.nodes:
    DG.nodes[node]['color'] = n_colors[node]

In [75]:
list(DG.nodes(data=True))

[('0', {'color': 'brown'}),
 ('1', {'color': 'palevioletred'}),
 ('2', {'color': 'orange'}),
 ('3', {'color': 'darkorchid'}),
 ('4', {'color': 'aliceblue'}),
 ('5', {'color': 'mintcream'}),
 ('6', {'color': 'lightsteelblue'})]

In [76]:
dnet = Network("400px","400px",notebook=True,directed=True)
dnet.from_nx(DG)
for edge in dnet.edges:
    edge['color'] = e_colors[f"{edge['from']}-{edge['to']}"]
for node in dnet.nodes:
    node['color'] = {'background': n_colors[node['id']], 'border': 'black'}
    node['shape'] = n_shapes[node['id']]

In [77]:
out_file = PATHS['html'] / f'graph_N{N:d}s{s:d}.html'
dnet.toggle_physics(True)
dnet.show(str(out_file))

/home/carlos/Documents/Information/edgar_german/repo/alternation_EFP/data/html/graph_N7s5.html
